In [ ]:
# install dependencies
!uv sync --all-extras

In [2]:
# load a COCO-pretrained YOLO11n model
from ultralytics import YOLO
model = YOLO("weights/yolo11n.pt")

In [3]:
# load database from KaggleHub
import kagglehub
path = adilshamim8_rock_paper_scissors_path = kagglehub.dataset_download('adilshamim8/rock-paper-scissors')
print("Path to dataset files:", path)

/Users/cheng/Projects/school/machine_learning/final/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/cheng/.cache/kagglehub/datasets/adilshamim8/rock-paper-scissors/versions/1


In [4]:
import os
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
import shutil

# 設定原始資料位置
DATA_DIR = os.path.join(path, 'train', 'train')
ANNOTATION_FILE = os.path.join(DATA_DIR, '_annotations.csv')
YOLO_DATASET_DIR = 'data/yolov11_rps_dataset'
os.makedirs(YOLO_DATASET_DIR, exist_ok=True)

In [6]:
# read annotation CSV
df = pd.read_csv(ANNOTATION_FILE)

# class mapping
class_map = {'Rock': 0, 'Paper': 1, 'Scissors': 2}

# split train/val
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

def convert_and_save(df_split, split_name):
    image_dir = os.path.join(YOLO_DATASET_DIR, 'images', split_name)
    label_dir = os.path.join(YOLO_DATASET_DIR, 'labels', split_name)
    os.makedirs(image_dir, exist_ok=True)
    os.makedirs(label_dir, exist_ok=True)

    for _, row in df_split.iterrows():
        filename = row['filename']
        label = row['class']
        xmin, ymin, xmax, ymax = row[['xmin', 'ymin', 'xmax', 'ymax']]

        img_path = os.path.join(DATA_DIR, filename)
        img = Image.open(img_path)
        w, h = img.size

        # 轉為 YOLO 格式 (中心 x/y, 寬, 高，全都除以圖像寬高)
        x_center = (xmin + xmax) / 2 / w
        y_center = (ymin + ymax) / 2 / h
        box_width = (xmax - xmin) / w
        box_height = (ymax - ymin) / h

        label_id = class_map[label]
        yolo_line = f"{label_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}"

        # 複製圖檔 + 儲存標註檔
        shutil.copy(img_path, os.path.join(image_dir, filename))
        with open(os.path.join(label_dir, filename.replace('.jpg', '.txt')), 'a') as f:
            f.write(yolo_line + '\n')

# convert the annotation to yolo format and save train/val splits
convert_and_save(train_df, 'train')
convert_and_save(val_df, 'val')
print("Format conversion complete")

Format conversion complete


In [15]:
DATA_YAML_PATH = "data/yolov11_rps_dataset/data.yaml"

# create YOLO training configuration
data_yaml = '''\
train: images/train
val: images/val

nc: 3
names: ['Rock', 'Paper', 'Scissors']
'''
with open(DATA_YAML_PATH, 'w') as f:
    f.write(data_yaml)

print("data.yaml created")

data.yaml created


In [18]:
results = model.train(data=DATA_YAML_PATH, epochs=5, imgsz=640, device="mps")  # remove device="mps" if not using Apple Silicon

Ultralytics 8.3.150 🚀 Python-3.13.2 torch-2.7.0 MPS (Apple M3)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/yolov11_rps_dataset/data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=weights/yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train8, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, 

train: Scanning /Users/cheng/Projects/school/machine_learning/final/data/yolov11_rps_dataset/labels/train.cache... 3272 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3272/3272 [00:00<?, ?it/s]

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 120.4±37.8 MB/s, size: 29.0 KB)



val: Scanning /Users/cheng/Projects/school/machine_learning/final/data/yolov11_rps_dataset/labels/val.cache... 898 images, 0 backgrounds, 0 corrupt: 100%|██████████| 898/898 [00:00<?, ?it/s]

Plotting labels to /Users/cheng/Projects/school/machine_learning/final/runs/detect/train8/labels.jpg... 


...

WARNING ⚠️ NMS time limit 3.600s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  93%|█████████▎| 27/29 [01:56<00:06,  3.03s/it]

WARNING ⚠️ NMS time limit 3.600s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [02:09<00:00,  4.48s/it]


                   all        898        922      0.583      0.497       0.47      0.282
                  Rock        407        421      0.619       0.52      0.489      0.303
                 Paper        257        257      0.502      0.482      0.463      0.264
              Scissors        241        244      0.629      0.488      0.459      0.279
Speed: 9.0ms preprocess, 27.3ms inference, 0.0ms loss, 77.2ms postprocess per image
Results saved to /Users/cheng/Projects/school/machine_learning/final/runs/detect/train8


In [19]:
model.export()

Ultralytics 8.3.150 🚀 Python-3.13.2 torch-2.7.0 CPU (Apple M3)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from '/Users/cheng/Projects/school/machine_learning/final/runs/detect/train8/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (5.2 MB)

TorchScript: starting export with torch 2.7.0...
TorchScript: export success ✅ 1.4s, saved as '/Users/cheng/Projects/school/machine_learning/final/runs/detect/train8/weights/best.torchscript' (10.4 MB)

Export complete (1.6s)
Results saved to /Users/cheng/Projects/school/machine_learning/final/runs/detect/train8/weights
Predict:         yolo predict task=detect model=/Users/cheng/Projects/school/machine_learning/final/runs/detect/train8/weights/best.torchscript imgsz=640  
Validate:        yolo val task=detect model=/Users/cheng/Projects/school/machine_learning/final/runs/detect/train8/weights/best.torchscript imgsz=640 data=data/yolov11_rps_dataset/d

'/Users/cheng/Projects/school/machine_learning/final/runs/detect/train8/weights/best.torchscript'